<a href="https://colab.research.google.com/github/SubbulakshmiSN/Retail_Stor_Performance_Analysis/blob/main/Sales_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *`IMPORT LIBRARIES`*

# *`READ THE DATA`*

In [7]:
import pandas as pd
import numpy as np

#Visualization
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


## *`SALES DATASET`*

In [8]:
sales= pd.read_csv("/content/sales_data_set.csv")
stores= pd.read_csv("/content/stores_data_set.csv")
features=pd.read_csv("/content/Features_data_set.csv")

In [9]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [10]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


## *`STORES DATASET`*

In [11]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [12]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


## *`FEATURES DATASET`*

In [13]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [14]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


# *`DATA HANDLING`*

In [15]:
sales.Date = pd.to_datetime(sales.Date, format="%d/%m/%Y")

In [16]:
features.Date = pd.to_datetime(features.Date, format="%d/%m/%Y")

In [17]:
sales.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [18]:
stores.isna().sum()

Store    0
Type     0
Size     0
dtype: int64

In [19]:
features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [20]:
''' features dataset has null values in the form of NaN
  so we will replace the NaN values with 0 '''

features.MarkDown1.fillna(0, inplace=True)
features.MarkDown2.fillna(0, inplace=True)
features.MarkDown3.fillna(0, inplace=True)
features.MarkDown4.fillna(0, inplace=True)
features.MarkDown5.fillna(0, inplace=True)
# features.CPI.fillna(0, inplace=True)
# features.Unemployment.fillna(0, inplace=True)

# *`EXPLORATORY DATA ANALYSIS`*

In [21]:
#1.Identify average customer visit in the type B store in April Months

#join the sales and store table
Store_merged=pd.merge(sales,stores,how='left',on='Store')
feature_merged=pd.merge(Store_merged,features,how='left',on=['Store','Date'])

#filter the type B store
typeB=feature_merged[(feature_merged['Type']=='B') & (feature_merged['Date'].dt.month==4)]

#calculate the average
# average_customer_visit=typeB.groupby('Store')['Weekly_Sales'].nunique().mean()
average_customer_visit=round(len(typeB)/len(typeB['Store'].unique()),2)
print("Average customer visit in the type B store in April Months : ",average_customer_visit)

Average customer visit in the type B store in April Months :  941.59


In [22]:
#2. Identify best average sales in holiday week for all store types

#filter the holiday week(isholiday=True)
holiday_week= Store_merged[Store_merged['IsHoliday']==True]

#calculate the average
average_sales_by_type = holiday_week.groupby('Type')['Weekly_Sales'].mean()

#identify the best average sales
best_store_type = average_sales_by_type.idxmax()
best_average_sales = average_sales_by_type.max()
print("Best average sales in holiday week for all store types : ",best_average_sales)
print("Best store type : ",best_store_type)

Best average sales in holiday week for all store types :  21297.51782416859
Best store type :  A


In [23]:
#3. Which store had a worst sales in leap year

#filter the leap year
leap_year= sales[sales['Date'].dt.is_leap_year]

#calculate the worst sales
worst_store = leap_year.groupby('Store')['Weekly_Sales'].sum().idxmin()
worst_sales = leap_year.groupby('Store')['Weekly_Sales'].sum().min()
print("Worst sales in leap year : ",worst_sales)
print("Worst store : ",worst_store)


Worst sales in leap year :  11435551.03
Worst store :  33


In [24]:
#4. What is the expected sales of each department when unemployment factor is greater>8

#join the sales and features table
sales_merged=pd.merge(sales,features,how='left',on=['Store','Date'])

#filter the expected sales
unemployment_factor = sales_merged[sales_merged['Unemployment']>8]
expected_sales = unemployment_factor.groupby('Dept')['Weekly_Sales'].mean()
print("Expected sales of each department when unemployment factor is greater>8 : ")
print(expected_sales)

Expected sales of each department when unemployment factor is greater>8 : 
Dept
1     17719.577853
2     43370.044649
3     10984.784400
4     25040.236272
5     19914.917085
          ...     
95    68200.568373
96    15487.837565
97    13024.744137
98     6225.629719
99      342.380187
Name: Weekly_Sales, Length: 81, dtype: float64


In [25]:
#5. Aggregate the net(total) sales of each department on month wise

# extract the month
sales['Month']=sales['Date'].dt.month

# aggregate the net(total) sales
net_sales=sales.groupby(['Month','Dept'])['Weekly_Sales'].sum()
print("Aggregate the net(total) sales of each department on month wise : ")
print(net_sales)

Aggregate the net(total) sales of each department on month wise : 
Month  Dept
1      1        4919530.75
       2       14344921.31
       3        4013980.96
       4        9132939.99
       5        6014721.65
                  ...     
12     95      28788259.55
       96       5312461.33
       97       6035160.16
       98       3323336.51
       99        152432.68
Name: Weekly_Sales, Length: 962, dtype: float64


In [26]:
#6. Which store performs high sales in week wise

# Extract week number from 'Date' column
sales['Week'] = sales['Date'].dt.strftime('%U')

# Grouping by store and week, then calculating the sum of weekly sales
weekly_sales_by_store = sales.groupby(['Store', 'Week'])['Weekly_Sales'].sum().reset_index()

# Finding the store with the highest weekly sales
store_with_highest_sales = weekly_sales_by_store.loc[weekly_sales_by_store.groupby('Week')['Weekly_Sales'].idxmax()]

print("Store with the highest sales in each week : ")
print(store_with_highest_sales.to_string(index=False))

Store with the highest sales in each week : 
 Store Week  Weekly_Sales
     4   01    3910242.34
     4   02    3807179.07
     4   03    3891491.70
     4   04    3742961.36
    20   05    6956061.74
    20   06    6783474.32
     4   07    6793995.99
     4   08    6230485.70
    20   09    6411461.09
    20   10    6141173.01
     4   11    6228872.18
    14   12    5823295.21
    20   13    6408110.05
    14   14    6672602.42
    20   15    6088586.98
    14   16    6270292.74
    14   17    5797002.71
    14   18    6415070.74
    20   19    6238057.26
     4   20    6126530.89
    14   21    6338716.01
    20   22    6528992.63
    14   23    6481631.92
    20   24    6322255.19
    13   25    6073876.33
    20   26    6252794.79
    20   27    6589128.94
    20   28    6206408.03
     4   29    6060221.86
     4   30    5834925.33
     4   31    6242210.10
     4   32    6306087.18
     4   33    6513613.49
    13   34    6002981.06
    20   35    6325808.95
    14   36    6299

In [27]:
#7. Identify better department performance based on the store on all the week

#group store and department wise sales
store_dept_sales=sales.groupby(['Store','Dept'])['Weekly_Sales'].sum()
#identify better department performance
best_dept_indices = store_dept_sales.groupby(level='Store').idxmax()


# Extract the corresponding weekly sales for the best department in each store
best_dept_sales = store_dept_sales.loc[best_dept_indices]

print("Better department performance based on the store on all the week : ")
print(best_dept_sales)

Better department performance based on the store on all the week : 
Store  Dept
1      92      19370632.64
2      92      23572153.03
3      38      15529566.07
4      92      22789210.43
5      38       7893570.24
6      92      14160545.90
7      72       6447844.96
8      95       9002059.31
9      38      11184287.78
10     72      20410926.56
11     95      11080680.91
12     2       10652763.01
13     92      23170876.20
14     92      26101497.71
15     38       7517821.66
16     38       7447254.23
17     38      11059588.03
18     38       9339684.03
19     92      16261990.45
20     92      23542625.04
21     2        6832625.68
22     38      10015536.66
23     72      10988695.25
24     92      17429136.57
25     38       6381753.99
26     38      14800243.10
27     92      20952094.22
28     92      14083635.33
29     38       6075233.36
30     38       8761108.01
31     92      18162446.96
32     92      14317578.85
33     92       4894555.79
34     95       9902061.88
35

In [29]:
#8. Identify the store which has minimum fuel price based on the week

# Group by date and store, then find the record with the minimum fuel price on each date
min_fuel_price_by_store_date = features.groupby(['Date', 'Store'])['Fuel_Price'].min().reset_index()

# Find the store with the minimum fuel price based on each date
store_with_min_fuel_price = min_fuel_price_by_store_date.loc[min_fuel_price_by_store_date.groupby('Date')['Fuel_Price'].idxmin()]

print("Store with the minimum fuel price based on each date:")
print(store_with_min_fuel_price)

Store with the minimum fuel price based on each date:
           Date  Store  Fuel_Price
35   2010-02-05     36       2.545
80   2010-02-12     36       2.539
125  2010-02-19     36       2.472
170  2010-02-26     36       2.520
215  2010-03-05     36       2.574
...         ...    ...         ...
8000 2013-06-28     36       3.428
8013 2013-07-05      4       3.385
8058 2013-07-12      4       3.368
8135 2013-07-19     36       3.507
8148 2013-07-26      4       3.580

[182 rows x 3 columns]


In [34]:
#9. Identify overall performance of the store based on year wise

#extract year from date
sales['Year'] = sales['Date'].dt.year

#group store and date wise sales
store_date_sales=sales.groupby(['Store','Year'])['Weekly_Sales'].sum().reset_index()

#identify overall performance based on weekly sales
store_overall_performance = store_date_sales.groupby('Store')['Weekly_Sales'].sum()


print("Overall performance of the store based on year wise : ")
print(store_overall_performance)

Overall performance of the store based on year wise : 
Store
1     2.224028e+08
2     2.753824e+08
3     5.758674e+07
4     2.995440e+08
5     4.547569e+07
6     2.237561e+08
7     8.159828e+07
8     1.299512e+08
9     7.778922e+07
10    2.716177e+08
11    1.939628e+08
12    1.442872e+08
13    2.865177e+08
14    2.889999e+08
15    8.913368e+07
16    7.425243e+07
17    1.277821e+08
18    1.551147e+08
19    2.066349e+08
20    3.013978e+08
21    1.081179e+08
22    1.470756e+08
23    1.987506e+08
24    1.940160e+08
25    1.010612e+08
26    1.434164e+08
27    2.538559e+08
28    1.892637e+08
29    7.714155e+07
30    6.271689e+07
31    1.996139e+08
32    1.668192e+08
33    3.716022e+07
34    1.382498e+08
35    1.315207e+08
36    5.341221e+07
37    7.420274e+07
38    5.515963e+07
39    2.074455e+08
40    1.378703e+08
41    1.813419e+08
42    7.956575e+07
43    9.056544e+07
44    4.329309e+07
45    1.123953e+08
Name: Weekly_Sales, dtype: float64


In [38]:
#10. Identify the performance of the store on week wise with without offers

# Merge sales data with features data on the 'Date' column
merged_data = pd.merge(sales, features, on='Date')

#checking the markdown values are not nan
merged_data['Offers_Present'] = ~merged_data[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].isnull().all(axis=1)

# Group by week and whether offers were present or not, then calculate the sum of weekly sales
performance_by_week_with_offers = merged_data.groupby(['Date', 'Offers_Present'])['Weekly_Sales'].sum().reset_index()

print("Performance of the store on week wise with without offers : ")
print(performance_by_week_with_offers)

Performance of the store on week wise with without offers : 
          Date  Offers_Present  Weekly_Sales
0   2010-02-05            True  2.238783e+09
1   2010-02-12            True  2.175150e+09
2   2010-02-19            True  2.172465e+09
3   2010-02-26            True  1.978586e+09
4   2010-03-05            True  2.109216e+09
..         ...             ...           ...
138 2012-09-28            True  1.968070e+09
139 2012-10-05            True  2.140499e+09
140 2012-10-12            True  2.075783e+09
141 2012-10-19            True  2.030508e+09
142 2012-10-26            True  2.049485e+09

[143 rows x 3 columns]
